In [1]:
import pandas as pd
import numpy as np
import re
import pickle
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

In [2]:
test_df = pd.read_csv('./src/dongjack_review.csv')
test_df = test_df[['name','review']]
print('리뷰 학습 데이터 크기:',test_df.shape)
test_df.head()

리뷰 학습 데이터 크기: (13650, 2)


,name,review
0,역전우동0410 동작남성점,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...
1,역전우동0410 동작남성점,1년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐.. 전 차...
2,역전우동0410 동작남성점,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요..하필 집근처지점이라 더 화나네요
3,역전우동0410 동작남성점,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠…못 앉을 곳 앉았나요...
4,역전우동0410 동작남성점,"맛없어요.,,. 배고파서 먹었다"


In [3]:
test_df['review'] = test_df['review'].str.replace('[^ 가-힣]',' ').str.replace(' +',' ')
print('리뷰 학습 데이터 크기:',test_df.shape)
test_df.head()

리뷰 학습 데이터 크기: (13650, 2)


,name,review
0,역전우동0410 동작남성점,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...
1,역전우동0410 동작남성점,년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐 전 차별하...
2,역전우동0410 동작남성점,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요 하필 집근처지점이라 더 화나네요
3,역전우동0410 동작남성점,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠 못 앉을 곳 앉았나요...
4,역전우동0410 동작남성점,맛없어요 배고파서 먹었다


In [4]:
from konlpy.tag import Okt

tag_list = ['Noun','Verb','Adjective','VerbPrefix']
okt = Okt()
tokenized_data = []  
for i in range(test_df.shape[0]):
    tokenized_sentence = okt.pos(str(test_df['review'][i]), stem=True) # 토큰화
    tag_checked_sentence = []
    for j in tokenized_sentence:
        x,y = j
        if y in tag_list:
            tag_checked_sentence.append(x)
    print(f'\r{i+1}개 형태소분석 완료',end='')
    tokenized_data.append(tag_checked_sentence)
for i in tokenized_data:
    for j in range(len(i)):
        i[j] = "'"+i[j]+"'"
test_df['토큰화댓글'] = tokenized_data
print('리뷰 학습 데이터 크기:',test_df.shape)
test_df.head()

13650개 형태소분석 완료리뷰 학습 데이터 크기: (13650, 3)


,name,review,토큰화댓글
0,역전우동0410 동작남성점,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...,"['포장', '주문', '하다', '집', '오다', '확인', '하다', '상품'..."
1,역전우동0410 동작남성점,년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐 전 차별하...,"['년', '쯤', '일', '하다', '이모', '한국', '중국', '바뀌다',..."
2,역전우동0410 동작남성점,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요 하필 집근처지점이라 더 화나네요,"['역전', '우동', '곳', '다니다', '보다', '여기', '제일', '맛없..."
3,역전우동0410 동작남성점,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠 못 앉을 곳 앉았나요...,"['자마자', '가위', '좌우', '휘저', '면서', '자리', '옮기다', '..."
4,역전우동0410 동작남성점,맛없어요 배고파서 먹었다,"['맛없다', '배고프다', '먹다']"


In [10]:
test_df = test_df[test_df['토큰화댓글'].str.len() > 3]
test_df = test_df.reset_index(drop=True) 
print('리뷰 학습 데이터 크기:',test_df.shape)
test_df.head()

리뷰 학습 데이터 크기: (9662, 4)


,name,review,토큰화댓글,score
0,역전우동0410 동작남성점,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...,"['포장', '주문', '하다', '집', '오다', '확인', '하다', '상품'...",0.010379
1,역전우동0410 동작남성점,년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐 전 차별하...,"['년', '쯤', '일', '하다', '이모', '한국', '중국', '바뀌다',...",0.038175
2,역전우동0410 동작남성점,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요 하필 집근처지점이라 더 화나네요,"['역전', '우동', '곳', '다니다', '보다', '여기', '제일', '맛없...",0.005100
3,역전우동0410 동작남성점,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠 못 앉을 곳 앉았나요...,"['자마자', '가위', '좌우', '휘저', '면서', '자리', '옮기다', '...",0.123762
4,한강치킨수제전문점 본점,튀김옷 얇고 쫄깃함,"['튀김', '옷', '얇다', '쫄깃함']",0.002647


In [11]:
import pickle

with open('./src/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
tokenized_data = test_df['토큰화댓글']
test = tokenizer.texts_to_sequences(tokenized_data)
test = pad_sequences(test, maxlen=20)
test

array([[4190,  198, 1035, ..., 1404,    2,   29],
       [ 123,    2,  638, ...,  540,    2,  666],
       [   0,    0,    0, ...,  471,   46,  959],
       ...,
       [   0,    0,    0, ...,   58,    2,   21],
       [   0,    0,    0, ...,    5,   94,    5],
       [ 150,  529, 1805, ..., 1092, 1183,  289]])

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

model = load_model('./src/model.h5')
pred = model.predict(test)
test_df['score'] = pred
df = test_df[['name','score']]
df

In [13]:
test_df

,name,review,토큰화댓글,score
0,역전우동0410 동작남성점,포장주문 했는데 집에 와서 확인하니 상품이 누락되어있어 연락하니 와서 찾아가라고 그...,"['포장', '주문', '하다', '집', '오다', '확인', '하다', '상품'...",0.010379
1,역전우동0410 동작남성점,년전쯤인가 일하시는 이모님 한국분에서 중국분으로 바뀌셨는데 맛이 없어짐 전 차별하...,"['년', '쯤', '일', '하다', '이모', '한국', '중국', '바뀌다',...",0.038175
2,역전우동0410 동작남성점,역전우동 수십곳을 다녀봤는데 여기가 제일 맛없어요 하필 집근처지점이라 더 화나네요,"['역전', '우동', '곳', '다니다', '보다', '여기', '제일', '맛없...",0.005100
3,역전우동0410 동작남성점,오자마자 가위를 좌우로 휘저으면서 자리 옮기라고 하는건 아니죠 못 앉을 곳 앉았나요...,"['자마자', '가위', '좌우', '휘저', '면서', '자리', '옮기다', '...",0.123762
4,한강치킨수제전문점 본점,튀김옷 얇고 쫄깃함,"['튀김', '옷', '얇다', '쫄깃함']",0.002647
...,...,...,...,...
9657,흥부왕족발부흥보쌈,전화해서 남자가 주문하면 두껍게 썰려오고 여자가 주문하면 대패삽겹마냥 썰어주네요 한...,"['전화하다', '남자', '주문', '하다', '두껍다', '썰리다', '여자',...",0.232865
9658,흥부왕족발부흥보쌈,존맛임 이집 항상 사람들 바글바글 다 이유가 있슴,"['존맛임', '이집', '항상', '사람', '이유', '있슴']",0.968287
9659,흥부왕족발부흥보쌈,아니 족발 왜이래요 지방 프로에 살 프로 이곳은 추천하지 않습니다,"['아니다', '족발', '왜', '요', '지방', '프로', '살', '프로',...",0.085185
9660,흥부왕족발부흥보쌈,족발은 뻑뻑함 맛없음 근데 같이 나오는 국물이 예술임 그것때문에 먹음 점심식사는 맛...,"['족발', '뻑뻑', '함', '맛없다', '나오다', '국물', '예술', '임...",0.581185


In [15]:
df = test_df[['name','score']]
df

,name,score
0,역전우동0410 동작남성점,0.010379
1,역전우동0410 동작남성점,0.038175
2,역전우동0410 동작남성점,0.005100
3,역전우동0410 동작남성점,0.123762
4,한강치킨수제전문점 본점,0.002647
...,...,...
9657,흥부왕족발부흥보쌈,0.232865
9658,흥부왕족발부흥보쌈,0.968287
9659,흥부왕족발부흥보쌈,0.085185
9660,흥부왕족발부흥보쌈,0.581185
